In [9]:
from feature import iProtein
from autogluon.tabular import TabularDataset,TabularPredictor
import pandas as pd
class automl():
    def __init__(self,train_data_path,test_data_path):
        self.train_data_path = train_data_path
        self.test_data_path = test_data_path
        self.feature_method = ['AAC', 'EAAC', 'CKSAAP type 1', 'CKSAAP type 2', 'DPC type 1', 'DPC type 2',
                                'DDE', 'TPC type 1', 'TPC type 2', 'binary', 'binary_6bit', 'binary_5bit type 1', 'binary_5bit type 2',
                                'binary_3bit type 1', 'binary_3bit type 2', 'binary_3bit type 3', 'binary_3bit type 4', 'binary_3bit type 5',
                                'binary_3bit type 6', 'binary_3bit type 7', 'AESNN3', 'GAAC', 'EGAAC', 'CKSAAGP type 1', 'CKSAAGP type 2',
                                'GDPC type 1', 'GDPC type 2', 'GTPC type 1', 'GTPC type 2', 'AAIndex', 'ZScale', 'BLOSUM62', 'NMBroto',
                                'Moran', 'Geary', 'CTDC', 'CTDT', 'CTDD', 'CTriad', 'KSCTriad', 'SOCNumber', 'QSOrder', 'PAAC', 'APAAC',
                                'OPF_10bit', 'OPF_7bit type 1', 'OPF_7bit type 2', 'OPF_7bit type 3', 'ASDC',
                                'DistancePair', 'AC', 'CC', 'ACC', 'PseKRAAC type 1', 'PseKRAAC type 2', 'PseKRAAC type 3A', 'PseKRAAC type 3B',
                                'PseKRAAC type 4', 'PseKRAAC type 5', 'PseKRAAC type 6A', 'PseKRAAC type 6B', 'PseKRAAC type 6C', 'PseKRAAC type 7', 
                                'PseKRAAC type 8', 'PseKRAAC type 9', 'PseKRAAC type 10', 'PseKRAAC type 11', 'PseKRAAC type 12', 'PseKRAAC type 13',
                                'PseKRAAC type 14', 'PseKRAAC type 15', 'PseKRAAC type 16', 'KNN']

        self.train_seq_with_label =self.read_data_fromcsv(self.train_data_path)[["seq","label"]]
        self.test_seq_with_label =self.read_data_fromcsv(self.test_data_path)[["seq","label"]]
        self.binary_metric = ['accuracy','roc_auc',
                            'average_precision','precision',
                            'recall','log_loss']
        self.mult_metric = ['root_mean_squared_error','mean_squared_error','mean_absolute_error',
                            'median_absolute_error','r2','pearsonr']

        self.problem_type = None

        

    def read_data_fromcsv(self,path) -> pd.DataFrame:
        data = pd.read_csv(path)
        if len(set(data['label']))>2:
            self.problem_type='b'
        else: self.problem_type = 'm'
        return(pd.read_csv(path))


    def Feature_extraction(self,seqlist:list,method):
        protein = iProtein(seqlist)
        protein.get_descriptor(method)
        return(protein.encodings,protein.error_msg)

    def train(self,method):
        self.best_predictor = TabularPredictor(label="label",path='Model'+'/'+ method).fit(self.train_data)

    def test(self):
        if self.problem_type == 'b':ex_metrics = self.binary_metric
        else:ex_metrics = self.mult_metric
        self.test_result = self.best_predictor.leaderboard(self.test_data,extra_metrics=ex_metrics)

    def run(self):
        train_label = self.train_seq_with_label["label"]
        test_label = self.test_seq_with_label["label"]
        for method in self.feature_method:
            train_feature,error_msg = self.Feature_extraction(list(self.train_seq_with_label["seq"]),method)
            if train_feature is None:
                print(error_msg)

            else:
                self.train_data = train_feature
                
                self.train_data["label"] = list(train_label)
                self.train_data.to_csv("./feature/"+method+'.csv')
                self.train(method)

                
                test_feature,error_msg = self.Feature_extraction(list(self.test_seq_with_label["seq"]),method)
                self.test_data = test_feature
                self.test_data["label"] = list(test_label)
                self.test()
                self.test_result.to_csv("./results/"+method+".csv")


In [10]:
test1 = automl("./train.csv","./val.csv")

In [11]:
test1.run()

Beginning AutoGluon training ...
AutoGluon will save models to "Model/AAC\"
AutoGluon Version:  0.4.0
Python Version:     3.7.6
Operating System:   Windows
Train Data Rows:    263
Train Data Columns: 20
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1404.2187832132272, 0.0, 349.09114, 371.86522)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    814.86 MB
	Train Data (Original)  Memory Usage: 0.04 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify specia

[1000]	valid_set's rmse: 287.388


	-285.9745	 = Validation score   (root_mean_squared_error)
	2.71s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-289.0777	 = Validation score   (root_mean_squared_error)
	0.72s	 = Training   runtime
	0.47s	 = Validation runtime
Fitting model: CatBoost ...
	-284.9727	 = Validation score   (root_mean_squared_error)
	1.17s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-287.6754	 = Validation score   (root_mean_squared_error)
	0.86s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-276.1115	 = Validation score   (root_mean_squared_error)
	8.15s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-270.3743	 = Validation score   (root_mean_squared_error)
	0.41s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-272.6422	 = Validation score   (root_mean_squared_error)
	4.27s	 = Training   runtime
	0.02s	 = Validati

                  model  score_test  root_mean_squared_error  mean_squared_error  mean_absolute_error  median_absolute_error        r2  pearsonr   score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0               XGBoost -325.156653              -325.156653      -105726.849179          -259.766264            -205.353410  0.214412  0.470912 -270.374344        0.011033       0.017537   0.405526                 0.011033                0.017537           0.405526            1       True          9
1              LightGBM -337.671086              -337.671086      -114021.762027          -273.348802            -216.901321  0.152778  0.435729 -285.974481        0.058000       0.006003   2.713288                 0.058000                0.006003           2.713288            1       True          4
2         LightGBMLarge -343.130961              -343.130961      -117738.856384          -275

Beginning AutoGluon training ...
AutoGluon will save models to "Model/EAAC\"
AutoGluon Version:  0.4.0
Python Version:     3.7.6
Operating System:   Windows
Train Data Rows:    263
Train Data Columns: 7680
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1404.2187832132272, 0.0, 349.09114, 371.86522)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    803.34 MB
	Train Data (Original)  Memory Usage: 16.16 MB (2.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify sp

...

                 model  score_test  root_mean_squared_error  mean_squared_error  mean_absolute_error  median_absolute_error        r2  pearsonr   score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0              XGBoost -377.906922              -377.906922      -142813.641716          -315.925168            -307.337070 -0.061156  -0.11733 -370.874686        0.020270       0.007020   0.275673                 0.020270                0.007020           0.275673            1       True          7
1      RandomForestMSE -380.701478              -380.701478      -144933.615685          -319.994342            -304.063501 -0.076908  -0.11733 -371.324607        0.118035       0.202791   0.593036                 0.118035                0.202791           0.593036            1       True          3
2        ExtraTreesMSE -380.701478              -380.701478      -144933.615685          -319.994

	-375.232	 = Validation score   (root_mean_squared_error)
	0.38s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ...
	-375.232	 = Validation score   (root_mean_squared_error)
	0.42s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-374.9857	 = Validation score   (root_mean_squared_error)
	0.56s	 = Training   runtime
	0.27s	 = Validation runtime
Fitting model: CatBoost ...
	-375.1177	 = Validation score   (root_mean_squared_error)
	1.88s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-374.9857	 = Validation score   (root_mean_squared_error)
	0.58s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-372.9981	 = Validation score   (root_mean_squared_error)
	2.82s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost ...
	-373.4971	 = Validation score   (root_mean_squared_error)
	0.24s	 = Training   runtime
	0.0s	 = Validation runtime
F

[1000]	valid_set's rmse: 375.232


Fitting model: WeightedEnsemble_L2 ...
	-372.877	 = Validation score   (root_mean_squared_error)
	0.31s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 11.82s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("Model/PseKRAAC type 16\")


                 model  score_test  root_mean_squared_error  mean_squared_error  mean_absolute_error  median_absolute_error        r2  pearsonr   score_val  pred_time_test  pred_time_val  fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2 -344.641881              -344.641881      -118778.026240          -268.224757            -256.294933  0.117437  0.344925 -372.877046        0.151547       0.280437  3.875179                 0.009316                0.000998           0.308461            2       True         10
1       NeuralNetTorch -344.665917              -344.665917      -118794.594381          -268.036299            -255.570172  0.117314  0.344925 -372.877896        0.005998       0.004999  2.593323                 0.005998                0.004999           2.593323            1       True          8
2      NeuralNetFastAI -344.961735              -344.961735      -118998.598326          -269.868325

In [12]:
len(test1.feature_method)

73